In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.dataset import *
from fastai.dataloader import *
from fastai.conv_learner import *
from fastai.plots import *

In [7]:
PATH = '/home/ubuntu/workspace/data/iciar18/full_dataset/'

In [3]:
top_layers = [AdaptiveConcatPool2d(), Flatten(), 
            nn.BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True), 
            nn.Dropout(p = 0.6), nn.Linear(4096,512), nn.ReLU(), 
            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True), 
            nn.Dropout(p = 0.6), 
            nn.Linear(512,4), 
            nn.LogSoftmax()]

In [4]:
model = nn.Sequential(*list(resnet50().children())[:-2])
for k, l in enumerate(top_layers):
    model.add_module(str(7+k+1), l)

In [8]:
loaded_model = load_model(model, f'{PATH}models/multi_exp2_ps.6_95.h5')

In [16]:
arch=resnet50
sz=299
fn = f'{PATH}Photos/Invasive/iv011.tif'
trn_tfms, val_tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.05) # get transformations
im = open_image(fn)
im_val = val_tfms(im)

In [17]:
model.type(torch.cuda.FloatTensor)
model.eval()
x = torch.from_numpy(im_val).type(torch.cuda.FloatTensor)
# x = x.permute(0,3,1,2)
print(x.size())

torch.Size([3, 299, 299])


In [18]:
log_pred = model(Variable(x).unsqueeze(0))
log_pred

Variable containing:
-7.1667 -4.4559 -0.0126 -8.7580
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]

In [21]:
pred = np.argmax(log_pred.data.cpu().numpy())
devt_classes = ['Benign', 'InSitu', 'Invasive', 'Normal']
submit_classes = {'Normal':0, 'Benign':1, 'InSitu':2, 'Invasive':3}
f'class prediction: {submit_classes[devt_classes[pred]]} => {devt_classes[pred]}'

'class prediction: 3 => Invasive'